# Import Lib

In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from keras.models import Model
from keras.layers import *
from six.moves import range

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

# Data Generation

In [9]:
questions = []
expected = []
seen = set()
print('Generating data...')

# Adder
while len(questions) < TRAINING_SIZE/2:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
    
#clear the seen data for subtract    
seen.clear()

# Subtracter
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # if a<b, swap the value
    if a<b:
        temp=a
        a=b
        b=temp
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
    
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [12]:
print(questions[:5], expected[:5])

['12+74  ', '51+639 ', '153+44 ', '7+887  ', '198+2  '] ['86  ', '690 ', '197 ', '894 ', '200 ']


# Processing

In [13]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [14]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 13)
(18000, 4, 13)
Validation Data:
(2000, 7, 13)
(2000, 4, 13)
Testing Data:
(60000, 7, 13)
(60000, 4, 13)


In [15]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False  True False False False False
   False False]
  [False False False False False False False False False False False
   False  True]
  [False False False False False False False False False False  True
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False  True False False False False False False
   False False]
  [ True False False False False False False False False False False
   False False]
  [ True False False False False False False False False False False
   False False]]

 [[False False False False False False False False  True False False
   False False]
  [False False False False  True False False False False False False
   False False]
  [False False  True False False False False False False False False
   False False]
  [False False False False False  True False False False False False
   False False]
  [False False False False False False False False  Tru

# Build Model

In [16]:
x_train.shape[1:]

(7, 13)

In [17]:
print('Build model...')

##### Build your own model here ############
# input to LSTM should have shape (nb_of_samples, seq_len, features)
# 
def my_model(input_data):
    
    model = Sequential()
    model.add(layers.LSTM(HIDDEN_SIZE, input_shape=(input_data[1], input_data[2]), name="lstm_1") )
    model.add(layers.RepeatVector(4, name="repeat_vector"))
    model.add(layers.LSTM(HIDDEN_SIZE, return_sequences=True, name="lstm_2"))
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax'), name="time_distributed"))

    model.summary()
    
    return model

Build model...


In [18]:
x_train.shape[1]

7

In [19]:
model = my_model(x_train.shape)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 4, 13)             1677      
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training

In [20]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 233us/step - loss: 1.9523 - acc: 0.3171 - val_loss: 1.8056 - val_acc: 0.3471
Q 265-29  T 236  ☒ 14  
Q 727-404 T 323  ☒ 114 
Q 708+297 T 1005 ☒ 110 
Q 28+949  T 977  ☒ 110 
Q 700-287 T 413  ☒ 114 
Q 441-67  T 374  ☒ 14  
Q 704-34  T 670  ☒ 14  
Q 0+43    T 43   ☒ 14  
Q 896-496 T 400  ☒ 110 
Q 496+528 T 1024 ☒ 110 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 160us/step - loss: 1.7897 - acc: 0.3452 - val_loss: 1.7803 - val_acc: 0.3497
Q 711+9   T 720  ☒ 100 
Q 357-36  T 321  ☒ 126 
Q 380+25  T 405  ☒ 110 
Q 17+65   T 82   ☒ 130 
Q 807-6   T 801  ☒ 12  
Q 316-94  T 222  ☒ 126 
Q 21+736  T 757  ☒ 110 
Q 540-58  T 482  ☒ 126 
Q 205+390 T 595  ☒ 110 
Q 61-12   T 49   ☒ 12  

-------------------

18000/18000 [==============================] - 3s 182us/step - loss: 1.3009 - acc: 0.5191 - val_loss: 1.3202 - val_acc: 0.5025
Q 198+26  T 224  ☒ 274 
Q 993-249 T 744  ☒ 964 
Q 269-99  T 170  ☒ 224 
Q 890+403 T 1293 ☒ 1230
Q 638-2   T 636  ☒ 632 
Q 6+717   T 723  ☒ 714 
Q 969+912 T 1881 ☒ 1707
Q 317+20  T 337  ☒ 374 
Q 362-2   T 360  ☒ 333 
Q 84+626  T 710  ☒ 774 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 175us/step - loss: 1.2668 - acc: 0.5334 - val_loss: 1.3236 - val_acc: 0.4965
Q 645-7   T 638  ☒ 620 
Q 950-68  T 882  ☒ 890 
Q 31-19   T 12   ☒ 1   
Q 123+65  T 188  ☒ 160 
Q 991-449 T 542  ☒ 465 
Q 320+757 T 1077 ☒ 705 
Q 46-46   T 0    ☒ 1   
Q 0+180   T 180  ☒ 110 
Q 68+879  T 947  ☒ 800 
Q 44+267  T 311  ☒ 370 

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 167us/step - loss: 0.9917 - acc: 0.6346 - val_loss: 1.0523 - val_acc: 0.6032
Q 923-303 T 620  ☒ 697 
Q 517+722 T 1239 ☒ 1275
Q 652+300 T 952  ☒ 901 
Q 708+85  T 793  ☒ 894 
Q 77-73   T 4    ☒ 1   
Q 512-222 T 290  ☒ 271 
Q 480-66  T 414  ☒ 402 
Q 821-41  T 780  ☒ 778 
Q 212+537 T 749  ☒ 644 
Q 76+760  T 836  ☒ 834 

--------------------------------------------------
Iteration 29
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 171us/step - loss: 0.9775 - acc: 0.6392 - val_loss: 1.0440 - val_acc: 0.6034
Q 150-15  T 135  ☒ 131 
Q 809-308 T 501  ☒ 473 
Q 594+627 T 1221 ☒ 1225
Q 561+30  T 591  ☒ 597 
Q 112+6   T 118  ☑ 118 
Q 458+7   T 465  ☒ 470 
Q 973-737 T 236  ☒ 251 
Q 884-273 T 611  ☒ 517 
Q 253-35  T 218  ☒ 235 
Q 747-19  T 728  ☒ 727 

--------------------------------------------------
Iteration 30
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 166us/step - loss: 0.8022 - acc: 0.7087 - val_loss: 0.9328 - val_acc: 0.6441
Q 62+24   T 86   ☒ 87  
Q 874-768 T 106  ☒ 112 
Q 245-79  T 166  ☒ 162 
Q 856+0   T 856  ☒ 867 
Q 388+181 T 569  ☒ 681 
Q 639+180 T 819  ☒ 849 
Q 82-26   T 56   ☒ 54  
Q 848-198 T 650  ☒ 654 
Q 51-32   T 19   ☑ 19  
Q 7+548   T 555  ☒ 542 

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 166us/step - loss: 0.7881 - acc: 0.7134 - val_loss: 0.9120 - val_acc: 0.6561
Q 566+196 T 762  ☒ 775 
Q 958-44  T 914  ☒ 902 
Q 92-81   T 11   ☒ 14  
Q 813-298 T 515  ☒ 423 
Q 747-546 T 201  ☒ 109 
Q 963+435 T 1398 ☒ 1399
Q 981-70  T 911  ☒ 914 
Q 554-1   T 553  ☒ 552 
Q 164+661 T 825  ☒ 842 
Q 551-32  T 519  ☒ 513 

--------------------------------------------------
Iteration 44
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 163us/step - loss: 0.5925 - acc: 0.7856 - val_loss: 0.7836 - val_acc: 0.6939
Q 502+12  T 514  ☒ 524 
Q 994+101 T 1095 ☒ 1005
Q 476-3   T 473  ☒ 474 
Q 0+292   T 292  ☒ 293 
Q 9+647   T 656  ☑ 656 
Q 75+93   T 168  ☒ 167 
Q 613+39  T 652  ☒ 640 
Q 673-99  T 574  ☒ 572 
Q 254-152 T 102  ☒ 122 
Q 872-494 T 378  ☒ 385 

--------------------------------------------------
Iteration 57
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 168us/step - loss: 0.5840 - acc: 0.7872 - val_loss: 0.7770 - val_acc: 0.6954
Q 848-227 T 621  ☒ 617 
Q 283+736 T 1019 ☒ 900 
Q 669-325 T 344  ☒ 355 
Q 63+58   T 121  ☒ 120 
Q 205-84  T 121  ☒ 129 
Q 56+577  T 633  ☑ 633 
Q 787+743 T 1530 ☑ 1530
Q 41+186  T 227  ☒ 217 
Q 582+2   T 584  ☑ 584 
Q 582+94  T 676  ☑ 676 

--------------------------------------------------
Iteration 58
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 167us/step - loss: 0.3587 - acc: 0.8803 - val_loss: 0.5900 - val_acc: 0.7679
Q 926+46  T 972  ☑ 972 
Q 687-56  T 631  ☑ 631 
Q 198+26  T 224  ☑ 224 
Q 66+72   T 138  ☑ 138 
Q 47+533  T 580  ☒ 589 
Q 43+143  T 186  ☒ 177 
Q 65+439  T 504  ☑ 504 
Q 512-222 T 290  ☑ 290 
Q 432-94  T 338  ☑ 338 
Q 774+868 T 1642 ☒ 1633

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 170us/step - loss: 0.3461 - acc: 0.8855 - val_loss: 0.5980 - val_acc: 0.7589
Q 156-90  T 66   ☑ 66  
Q 17+65   T 82   ☑ 82  
Q 584-306 T 278  ☒ 376 
Q 366-29  T 337  ☒ 336 
Q 0+169   T 169  ☒ 170 
Q 209-69  T 140  ☒ 141 
Q 381+774 T 1155 ☒ 1186
Q 589+49  T 638  ☒ 648 
Q 540+18  T 558  ☑ 558 
Q 879+84  T 963  ☒ 973 

--------------------------------------------------
Iteration 72
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 164us/step - loss: 0.1941 - acc: 0.9476 - val_loss: 0.4674 - val_acc: 0.8258
Q 707-95  T 612  ☑ 612 
Q 684-6   T 678  ☑ 678 
Q 784+44  T 828  ☑ 828 
Q 36+880  T 916  ☑ 916 
Q 6+717   T 723  ☑ 723 
Q 484-22  T 462  ☑ 462 
Q 297-61  T 236  ☒ 226 
Q 754-655 T 99   ☒ 91  
Q 76+470  T 546  ☒ 556 
Q 21+49   T 70   ☒ 79  

--------------------------------------------------
Iteration 85
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 175us/step - loss: 0.1880 - acc: 0.9503 - val_loss: 0.4737 - val_acc: 0.8253
Q 211+66  T 277  ☒ 267 
Q 731+91  T 822  ☑ 822 
Q 102-63  T 39   ☒ 48  
Q 699+98  T 797  ☒ 788 
Q 196+79  T 275  ☒ 265 
Q 475-413 T 62   ☒ 44  
Q 558-3   T 555  ☒ 554 
Q 494+244 T 738  ☒ 718 
Q 254-152 T 102  ☑ 102 
Q 638+458 T 1096 ☒ 1075

--------------------------------------------------
Iteration 86
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 164us/step - loss: 0.1021 - acc: 0.9795 - val_loss: 0.4194 - val_acc: 0.8535
Q 942+7   T 949  ☒ 959 
Q 84+856  T 940  ☑ 940 
Q 548-221 T 327  ☑ 327 
Q 864-33  T 831  ☑ 831 
Q 431-30  T 401  ☑ 401 
Q 720-0   T 720  ☑ 720 
Q 786+971 T 1757 ☒ 1766
Q 347-36  T 311  ☑ 311 
Q 958-919 T 39   ☒ 38  
Q 716+37  T 753  ☑ 753 

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 162us/step - loss: 0.0957 - acc: 0.9816 - val_loss: 0.4085 - val_acc: 0.8544
Q 269+245 T 514  ☒ 505 
Q 848-198 T 650  ☒ 559 
Q 492-59  T 433  ☒ 423 
Q 958-373 T 585  ☑ 585 
Q 379+37  T 416  ☑ 416 
Q 522-171 T 351  ☒ 341 
Q 57+232  T 289  ☑ 289 
Q 756-433 T 323  ☒ 324 
Q 902-86  T 816  ☑ 816 
Q 56-55   T 1    ☒ 2   


# Testing

In [21]:
def vectorization(ary_x, ary_y):
    print('Vectorization...')
    x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
    y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(test_x):
        x[i] = ctable.encode(sentence, MAXLEN)
    for i, sentence in enumerate(test_y):
        y[i] = ctable.encode(sentence, DIGITS + 1)
    return x, y

In [22]:
def evaluation_test(x_test, y_test, model):
    x_val, y_val = vectorization(test_x, test_y)
    for i in range(len(y_val)):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

In [23]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
# test_x = ["555+175", "860+7  ", "340+29 "]
# test_y = ["730 ", "867 ", "369 "] 
# evaluation_test(test_x, test_y, model)
#####################################################

testing_callback = model.evaluate(x=test_x, y=test_y, batch_size=1, verbose=1)
for i in range(len(model.metrics_names)):
        print(model.metrics_names[i]+" : "+str(testing_callback[i]))

MSG : Prediction
60000/60000 [==============================] - 112s 2ms/step


[0.3925717580507774, 0.8617083333333333]